In [1]:
import os
import pandas as pd

dir0 = 'data0/'
dir1 = 'data1/'

def rename_columns(dir0, dir1):
    for file in os.listdir(dir0):
        if file.endswith('.csv'):
            df = pd.read_csv(dir0 + file)
            if file == 'policies.csv':
                df.to_json('json_files/' + file.replace('.csv', '.json'), orient='records')
                continue

            df.columns = df.columns.str.lower()
            df.rename(columns={'geo': 'country', 'time_period': 'year', 'obs_value': 'value'}, inplace=True)
            df.drop(columns=['dataflow', 'last update', 'freq', 'obs_flag', 'conf_status'], inplace=True, errors='ignore')

            df = df[~df['country'].isin(['Türkiye', 'Iceland', 'Switzerland', 'Norway', 'Montenegro', 'North Macedonia', 'Serbia', 'Albania', 'Bosnia and Herzegovina', 'Kosovo*', 'Liechtenstein', 'Moldova', 'United Kingdom'])]

            # Rename
            df['country'] = df['country'].replace('Czechia', 'Czech Republic')
            df['country'] = df['country'].replace('European Union - 27 countries (from 2020)', 'EU')
            df['country'] = df['country'].replace('European Union (aggregate changing according to the context)', 'EU')


            # Remove
            df = df[df['country'] != 'European Union - 28 countries (2013-2020)']
            df = df[df['country'] != 'Euro area – 20 countries (from 2023)']
            df = df[df['country'] != 'European Commission']
            df = df[df['country'] != 'European Investment Bank (EIB)']

            countries_eu = ['EU', 'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
            countries_not_eu = df[~df['country'].isin(countries_eu)]['country'].unique()
            if len(countries_not_eu) > 0:
                print(file, countries_not_eu)

            
            if 'unit' in df.columns:
                pivot_index = ['country', 'unit']
                main_columns = ['indicator', 'country', 'unit']
            else:
                pivot_index = ['country']
                main_columns = ['indicator', 'country']
                
            df_wide = df.pivot_table(index=pivot_index, columns='year', values='value', aggfunc='first').reset_index()

            df_wide['indicator'] = file.replace('.csv', '')
            
            year_columns = sorted([col for col in df_wide.columns if col not in main_columns])
            df_wide = df_wide[main_columns + year_columns]
            
            df_wide.to_csv(dir1 + file, index=False)
            df_wide.to_json('json_files/' + file.replace('.csv', '.json'), orient='records')

rename_columns(dir0, dir1)

In [20]:
import pandas as pd

# Import policies.csv
df = pd.read_csv('policies.csv')
df

,country,year,policy,description,status,sector,source
0,Austria,2005,EU Emission Trading Scheme (ETS),The objective is to limit the CO2 emission fro...,Implemented,Energy consumption; Energy supply; Industrial ...,https://discomap.eea.europa.eu/map/GHG_PAMS/Au...
1,Austria,1993,Domestic Environmental Support Scheme,The objective of the Domestic Environmental Su...,Implemented,Energy consumption; Energy supply; Transport,https://discomap.eea.europa.eu/map/GHG_PAMS/Au...
2,Austria,2007,Austrian Climate and Energy Fund,"In 2007, the Federal Government established a ...",Implemented,Energy supply; Transport,https://discomap.eea.europa.eu/map/GHG_PAMS/Au...
3,Austria,2002,Increase the share of renewable energy in powe...,Beyond the traditional use of large-scale hydr...,Implemented,Energy supply,https://discomap.eea.europa.eu/map/GHG_PAMS/Au...
4,Austria,2008,Increase energy efficiency in energy and manuf...,"Based on EU legislation, Austria has implement...",Implemented,Energy consumption; Energy supply,https://discomap.eea.europa.eu/map/GHG_PAMS/Au...
...,...,...,...,...,...,...,...
3027,Switzerland,2014,Investment aids,"Investment aids apply to photovoltaics, wind, ...",Implemented,Energy supply,https://discomap.eea.europa.eu/map/GHG_PAMS/Sw...
3028,Switzerland,2023,Non-volatile particle matter emission regulati...,Coated soot particle emissions are directly co...,Adopted,Transport,https://discomap.eea.europa.eu/map/GHG_PAMS/Sw...
3029,Switzerland,2025,Sustainable aviation fuel policy,Introduction of sustainable aviation fuel with...,Planned,Transport,https://discomap.eea.europa.eu/map/GHG_PAMS/Sw...
3030,Switzerland,2022,Obligations in relation to chemical conversion...,N2O generated as a by-product must be converte...,Implemented,Industrial processes,https://discomap.eea.europa.eu/map/GHG_PAMS/Sw...


In [ ]:
# Print all "sector" values, separated by semicolon
sectors = df['sector'].unique().tolist()
sectors.sort()

# Have only the first part before ;
df['sector'] = df['sector'].str.split(';').str[0]

print(df['sector'].unique())

['Energy consumption' 'Energy supply' 'Transport' 'Industrial processes'
 'Agriculture' 'LULUCF' 'Waste management/waste' 'Other sectors']


In [82]:
file_name = 'Water_exploitation_index'

# read first file from data1
df = pd.read_csv('data1/' + file_name + '.csv')

df['sector'] = 'Energy consumption'

df.head()

,indicator,country,unit,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,sector
0,Water_exploitation_index,Austria,Percentage,1.25,1.36,1.26,1.67,1.51,1.56,1.60,...,1.75,2.13,1.88,2.10,2.05,0.63,1.79,2.00,2.12,Energy consumption
1,Water_exploitation_index,Belgium,Percentage,7.12,5.62,6.32,10.47,11.75,10.94,7.68,...,3.86,3.74,3.55,5.26,4.22,4.21,3.82,3.27,5.19,Energy consumption
2,Water_exploitation_index,Bulgaria,Percentage,1.44,1.10,0.99,1.48,1.01,0.79,0.95,...,0.63,0.76,0.93,1.13,0.84,0.99,1.02,0.91,1.21,Energy consumption
3,Water_exploitation_index,Croatia,Percentage,0.18,0.17,0.15,0.16,0.15,0.18,0.18,...,0.10,0.15,0.13,0.21,0.20,0.20,0.25,0.24,0.26,Energy consumption
4,Water_exploitation_index,Cyprus,Percentage,59.54,63.81,72.14,70.20,76.76,75.42,69.11,...,68.05,71.61,71.19,70.86,69.17,66.74,71.08,71.18,71.04,Energy consumption


In [83]:
df.to_csv('data1/' + file_name + '.csv', index=False)